# torch - learning

## einsum

1. 自由索引，出现在箭头右边的索引，比如上面的例子就是 i 和 j；
2. 求和索引，只出现在箭头左边的索引，表示中间计算结果需要这个维度上求和之后才能得到输出，比如上面的例子就是 k；
   
三条规则
规则一，equation 箭头左边，在不同输入之间重复出现的索引表示，把输入张量沿着该维度做乘法操作，比如还是以上面矩阵乘法为例， "ik,kj->ij"，k 在输入中重复出现，所以就是把 a 和 b 沿着 k 这个维度作相乘操作；
规则二，只出现在 equation 箭头左边的索引，表示中间计算结果需要在这个维度上求和，也就是上面提到的求和索引；
规则三，equation 箭头右边的索引顺序可以是任意的，比如上面的 "ik,kj->ij" 如果写成 "ik,kj->ji"，那么就是返回输出结果的转置，用户只需要定义好索引的顺序，转置操作会在 einsum 内部完成。

In [1]:
import torch

tensor_i = torch.randn(4,4)
tensor_i
tensor_j = torch.randn(4,4)

k = torch.einsum('ii->i', tensor_i)# 对角元素求和

k = torch.einsum('ij,ij->',tensor_i,tensor_j)# 向量dot求和

k = torch.einsum("ij->i",tensor_i) #保留I的维度

k = torch.einsum("i,j->ij",tensor_i,tensor_j) #

# 本质上都是在相同的维度上进行运算，然后进行求和。

k = torch.einsum('')
k

from einops import rearrange, reduce #  两种可以重组不同torch的库

tensor([5.7030, 4.2929, 1.7279, 1.7012])

### broadcast mechanism
可广播的一对张量需满足以下规则：
- 每个张量至少有一个维度。
- 迭代维度尺寸时，从尾部的维度开始，维度尺寸或者相等，
​- 或者其中一个张量的维度尺寸为 1 ，
​- 或者其中一个张量不存在这个维度。


### pad 

从维度上作为逆序，首先在前面的维度中进行相关的操作。

In [15]:
# import beartype
from beartype import beartype
# from beartype import Tensor
# F.pad
import torch.nn.functional as F

import torch 
i = torch.full((2,3,4),3.0)

# 以上的做法可以把全部的值都当作元祖放置到gpu上面
j = F.pad(i,(2,0),mode='constant',value=0)


# masked_fill 的做法是，通过加入bool型变量的值，把在true的位置都填入pad_id 进行填充
j = j.masked_fill(j== j,0.)

tensor([[[0., 0., 3., 3., 3., 3.],
         [0., 0., 3., 3., 3., 3.],
         [0., 0., 3., 3., 3., 3.]],

        [[0., 0., 3., 3., 3., 3.],
         [0., 0., 3., 3., 3., 3.],
         [0., 0., 3., 3., 3., 3.]]])

In [ ]:
def get_arange_start_at_token_id(
        token_ids: torch.Tensor,
        token_id: int,
        pad_id = -1):
    is_token_id_mask = token_ids == token_id
    arange = (is_token_id_mask.cumsum(dim=1) > 0).cumsum(dim=-1)# cumsum是一种累积和的方式
    before_token_mask = arange == 0
    arange = arange - 1
    arange = arange.masked_fill(before_token_mask, pad_id)
    return arange # end 之前填充pad_id and after is [0,...]
    

def default_weight_fn(t):
    return (1. - t * 0.2).clamp(min=0.)
def weigh_and_mask(
        token_ids: torch.Tensor,
        token_id: int,
        pad_id = -1,
        weighting_fn: Callable = default_weight_fn):
    t = get_arange_start_at_token_id(token_ids, token_id, pad_id)
    weighs = weighting_fn(t)
    return weighs.masked_fill(t == pad_id, 0.)

    



### 对于张量的操作

#### gather

gather的想法是从tensor中按照标签选取合适的值。


原理是在通过扩散和src—data 中相同的维度然后通过把里面的数据替换到指定的维度中。


同时可以选择相应的行向量和列向量的形式。


In [10]:
import torch

tensor_0 = torch.arange(3,12).view(3,3)
index = torch.tensor([[2,1,0],[0,2,1]])# 
index = index.view(2,3)
tensor_1 = tensor_0.gather(0,index)
tensor_1 


tensor([[ 9,  7,  5],
        [ 3, 10,  8]])

In [ ]:
# pad_sequence 用于序列的填充


def pad_sequence(
    sequences: Union[Tensor, List[Tensor]],
    batch_first: bool = False,
    padding_value: float = 0.0,
) -> Tensor: 
pad_sequence = partial(pad_sequence, padding_value = 0) # 可以通过partial 来固定一部分参数。
# batch_first 表示是否把batch放在第一维度上面
# list 的模式是填充为统一的长度，同时增加batch的长度
# padding value 表示为填充的值
# 表示要么用tensor 要么用List tensor的模式


#### cumsum

cumsum 是 PyTorch 中的一个函数，它用于计算输入张量的累积和（cumulative sum）。这个函数会沿着指定的维度计算累积和。

在你的代码中，is_token_id_mask.cumsum(dim = -1) 这行代码计算的是 is_token_id_mask 张量在最后一个维度上的累积和。is_token_id_mask 是一个布尔张量，其中的每个元素表示相应的 token id 是否等于指定的 token_id。


In [13]:
is_token_id_mask = torch.tensor([[True, False, True],
                                 [False, True, False]], dtype=torch.bool)
(is_token_id_mask.cumsum(dim=-1) >0 ).cumsum(dim=-1) # 一个设置的开始的位置，对于后续数据进行测量

tensor([[1, 2, 3],
        [0, 1, 2]])

## regex    

In [20]:
import re
re.findall(r'\d+','1234')# 采用的是pattern的形式 和匹配字符串的形式
re.escape("hello world\n") # 表示为处理正规式的方式



def replace_fn(
        matches,
        registry,
        delimiter = '->'
):
    orig_text = matches.group(0)
    text_without_end_api_token = matches.group(1)
    end_api_token = matches.group(4)
    function_name = matches.group(2)

    if function_name not in registry:
        return orig_text
    fn = registry[function_name]
    params = matches.group(3).splits(',')
    params = list(map(lambda x: x.strip(), params))# 好帅的去括号方式
    params = list(filter(len,params))
    
    if any([])
    pass

filter(function,iterable) # filter 函数的使用方式,通过function判断可迭代变量中的式子是否为真，同时进行返回
regex = rf'hello'
text = 'hello'
re.sub(regex,replace_fn, text) # sub 通过自动匹配比较的方式，从text中获得match，然后传递参数把里面的

'hello\\ world\\\n'

## dataloader

```python
class DataLoader(object):
    def __init__(self, dataset, batch_size=1, shuffle=False, sampler=None,
                 batch_sampler=None, num_workers=0, collate_fn=default_collate,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None)
    
in : list(BatchSampler(SequentialSampler(range(10)), batch_size=3, drop_last=False))
out: [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]

```
以上可以看到两种sample的方式：sampler 和 batch_sampler，默认设置成None，同时batch_sampler的生成是基于sampler的同时按照batchsize进行分组。

- shuffle = True sampler=RandomSampler(dataset)
- shuffle = False 则sampler=SequentialSampler(dataset)
- _如果自定义了Sample_ shuffle需要设置成false
-  drop_last 是否能整除全部的数列

### collate_fn 

```python
class DataLoader(object): 
    ... 
     
    def __next__(self): 
        if self.num_workers == 0:   
            indices = next(self.sample_iter)  
            batch = self.collate_fn([self.dataset[i] for i in indices]) # this line 
            if self.pin_memory: 
                batch = _utils.pin_memory.pin_memory_batch(batch) 
            return batch
            
def prompt_collate_fn(data, padding_value = 0):# 应该是直接对于一批的数据进行同步处理
    prompts, prompt_lengths = zip(*data)
    prompts = pad_sequence(prompts, padding_value = padding_value)
    return prompts, torch.stack(prompt_lengths) 
```

如上代码所示，collate_fn把多个数据合称为一个batch数据
### sampler

sample的各种方法都是继承同一个父类，同时我们只需要负责iter函数的设计，同时他的返回值是可迭代的
```python
iter(range(len(self.data_source)))
```

In [ ]:
# 父类

class Sampler(object):
    r"""Base class for all Samplers.
    Every Sampler subclass has to provide an :meth:`__iter__` method, providing a
    way to iterate over indices of dataset elements, and a :meth:`__len__` method
    that returns the length of the returned iterators.
    .. note:: The :meth:`__len__` method isn't strictly required by
              :class:`~torch.utils.data.DataLoader`, but is expected in any
              calculation involving the length of a :class:`~torch.utils.data.DataLoader`.
    """

    def __init__(self, data_source):
        pass
    def __iter__(self):
        raise NotImplementedError
    def __len__(self):
        raise len(self.data_source)

In [16]:
from torch.utils.data import Dataset, DataLoader
from beartype import beartype
from beartype import Callable,Optional,Union,List,Tuple
import re

DataLoader(dataset,batchsize= 16,shuffle= True,drop_last = True)
class FinetuneData(Dataset):# 数据至少含有len和getitem两个函数
    def __init__(
        self,
        tokens : torch.Tensor
    ):
        self.tokens = tokens
    def __len__(self):
        return len(self.tokens)

    def __getitem(self, idx):
        return self.tokens[idx]
    

class PromptDataset(Dataset):
    def __init__(
            self,
            prompt:str,
            prompt_input_tag: str,
            data:List[str],
            tokenizer_encode: Callable  
    ):
        self.data = data
        self.prompt = prompt
        self.prompt_input_tag_regex = re.escape(prompt_input_tag)
        self.tokenizer_encode = tokenizer_encode
    def __len__(self):
        return len(self.data)
    
    def __getitem(self,idx):
        data_string = self.data[idx]
        data_with_prompt = re.sub(self.prompot_input_tag_regex,data_string,self.prompt)
        token_ids = self.tokenizer_encode(data_with_prompt)
        return torch.tensor(token_ids).long(), torch.tensor(len(token_ids)).long() # 一个作为label应该可以用作之后的处理



### loss

#### cross Entropy loss

$$H(p,q) = -\sum_x (p(x))\lg softmax(q(x)) $$

p 为真实的值， q为预测值


In [17]:
import torch
import torch.nn as nn 
import math
import numpy as np

entroy = nn.CrossEntropyLoss()
input=torch.Tensor([[0.1234, 0.5555,0.3211],[0.1234, 0.5555,0.3211],[0.1234, 0.5555,0.3211],])
target = torch.tensor([0,1,2])
loss = entroy(input,target)
loss


tensor(1.1142)

## learn from code

### tool-transformer

In [ ]:
mask[..., api_start_token_id] = True # ... 表示任意数量的冒号

last_logits = logits[batch_indices, position_indices] # 表示在维度上分别用两遍的值进行筛选

 t.clamp(min = eps).log()  # 小于 eps的值都填上eps ，同时.log（）表示对向量取ln

torch.zeros_like(j).uniform_(0, 1) # uniform 采用的正态分布的形式

tensor = tensor.type(torch.float) # 表示修改张量的形式

tensor.argmax(dim = ) # 返回的是指定维度数上面减一的操作，同时也是在指定的维度上进行统计
tensor.any(dim = -1) # 同样的，不过是检查bool型的变量


tokens, tokens_without_api_response, tokens_with_api_response = map(lambda t: t.to(device), (tokens, tokens_without_api_response, tokens_with_api_response))

tensor.topk(k,dim ) # 在相应的维度上进行操作，同时返回的indices 和 values 两个子元。

tensor.item() # 表示的是把tensor转化为标量

pad_sequence() #

tensor == p # 表示对于每一个维度都需要进行bool的判断 同时如果是完整的传入了一个向量，必须判断是完全相等的形式

torch.finfo(last_logits.dtype).max # 表示的寻找序列变量的极值

nametuple('name',['','']) # 创建命名类的元祖，可以通过名字来访问相应的标签。

zip(*data) # data是多种可迭代数据，zip把每个数据组的头元素压缩在一起成为元组，用list的形式表现出来

torch.stack() # 可以把一组向量堆叠起来



## python 语法

### dict

** dict 表示把键对的值用key= value的方式表现出来传递到函数中 


### function

```python

filter(filter_fn,iterable) 

any() # 函数接受一个可以迭代的对象，如果里面有任意一个元素为真，则返回true
```

## huggingface NLP 



### logits 

在NLP任务中，logits表示对于各个模型分类的概率，相对于softmax处理之前的形式，softmax处理之后的形式就是概率的形式。

In [ ]:
from transformers import AutoModelForSequenceClassification,tokenizer
clf = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 2)
inputs = tokenizer(raw_input,padding = True, truncation = True, return_tensors = 'pt')# input 可以作为list放进入
outputs = clf(**inputs) 

输出 ： dict_keys(['loss', 'logits', 'hidden_states', 'attentions'])
tensor([[-4.2098,  4.6444],
        [ 0.6367, -0.3753]], grad_fn=<AddmmBackward>)